# Import

In [ ]:
name = "resnetrs270_cv10_lr0002_batch32"

import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
import cv2
import pickle

import os
import timm
import random
from efficientnet_pytorch import EfficientNet

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda:1')

In [ ]:
train_png = sorted(glob('../data/train/*.png'))
test_png = sorted(glob('../data/test/*.png'))

train_y = pd.read_csv("../data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

# train_imgs = [img_load(m) for m in tqdm(train_png)]
# test_imgs = [img_load(n) for n in tqdm(test_png)]

# np.save('../data/train_imgs_384', np.array(train_imgs))
# np.save('../data/test_imgs_384', np.array(test_imgs))

In [ ]:
train_imgs = np.load('../data/train_imgs_384.npy')
test_imgs = np.load('../data/test_imgs_384.npy')

In [ ]:
# meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
# stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

# meanR = np.mean([m[0] for m in meanRGB])/255
# meanG = np.mean([m[1] for m in meanRGB])/255
# meanB = np.mean([m[2] for m in meanRGB])/255

# stdR = np.mean([s[0] for s in stdRGB])/255
# stdG = np.mean([s[1] for s in stdRGB])/255
# stdB = np.mean([s[2] for s in stdRGB])/255

# print("train 평균",meanR, meanG, meanB)
# print("train 표준편차",stdR, stdG, stdB)

In [ ]:
# meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
# stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

# meanR = np.mean([m[0] for m in meanRGB])/255
# meanG = np.mean([m[1] for m in meanRGB])/255
# meanB = np.mean([m[2] for m in meanRGB])/255

# stdR = np.mean([s[0] for s in stdRGB])/255
# stdG = np.mean([s[1] for s in stdRGB])/255
# stdB = np.mean([s[2] for s in stdRGB])/255

# print("test 평균",meanR, meanG, meanB)
# print("test 표준편차",stdR, stdG, stdB)

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
            ])
            img = train_transform(img)
        if self.mode == 'test':
            test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
            img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
            self.model = timm.create_model('resnetrs270', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
            self.model = timm.create_model('resnetrs270', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [ ]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [ ]:
pred_train_dict = {}
pred_test_dict = {}

In [ ]:
import gc

cv = StratifiedKFold(n_splits = 10, random_state = 2022, shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
#     print("----------fold_{} start!----------".format(idx))
    t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
    t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

    # Train
    train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

    # Val
    val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
    val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

    gc.collect()
    torch.cuda.empty_cache()
    best=0

    model = Network().to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()  

    best_f1 = 0
    early_stopping = 0
    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_y=[]
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)


            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()/len(train_loader)
            train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            train_y += y.detach().cpu().numpy().tolist()
        train_f1 = score_function(train_y, train_pred)
        state_dict= model.state_dict()
        model.eval()
        with torch.no_grad():
            val_loss = 0 
            val_pred = []
            val_y = []


            for batch in (val_loader):
                x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
                y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
                with torch.cuda.amp.autocast():
                    pred_val = model(x_val)
                loss_val = criterion(pred_val, y_val)

                val_loss += loss_val.item()/len(val_loader)
                val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
                val_y += y_val.detach().cpu().numpy().tolist()
            val_f1 = score_function(val_y, val_pred)
            print(f'fold{idx+1} epoch{epoch} score: {val_f1:.5f}')

            if val_f1 > best_f1:
                best_epoch = epoch
                best_loss = val_loss
                best_f1 = val_f1
                early_stopping = 0

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': optimizer.state_dict(),
                            'scaler': scaler.state_dict(),
                     }, f'../model/{name}_best_model_{idx+1}.pth')
#                 print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
            else:
                early_stopping += 1

                # Early Stopping
        if early_stopping == 20:
            TIME = time.time() - start
            print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
            print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
            print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
            break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

In [ ]:
pred_train = np.zeros((len(train_imgs), 88))
pred_test = np.zeros((len(test_imgs), 88))

test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
    print("----------fold_{} predict start!----------".format(idx+1))
    
    t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
    t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

    # Val
    val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)

    gc.collect()
    torch.cuda.empty_cache()

    model_test = Network(mode = 'test').to(device)
    model_test.load_state_dict(torch.load((f'../model/{name}_best_model_{idx+1}.pth'))['state_dict'])
    model_test.eval()
    
    pred_train_list = []
    with torch.no_grad():
        for batch in (val_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            with torch.cuda.amp.autocast():
                pred_train_local = model_test(x)
                pred_train_list.extend(pred_train_local.detach().cpu().numpy())
                
    gc.collect()
    torch.cuda.empty_cache()

    model_test = Network(mode = 'test').to(device)
    model_test.load_state_dict(torch.load((f'../model/{name}_best_model_{idx+1}.pth'))['state_dict'])
    model_test.eval()
            
    pred_test_list = []
    with torch.no_grad():
        for batch in (test_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            with torch.cuda.amp.autocast():
                pred_test_local = model_test(x)
                pred_test_list.extend(pred_test_local.detach().cpu().numpy())
                
    pred_train[val_idx, :] = pred_train_list
    pred_test += np.array(pred_test_list) / 10

In [ ]:
pred_train_dict[f'{name}_seed{str(2022)}'] = pred_train
pred_test_dict[f'{name}_seed{str(2022)}'] = pred_test

In [ ]:
def sort_dict(model, pred_dict, pred_test_dict):
    pred_dict_local = {}
    for key, value in pred_dict.items():
        if model in key:
            pred_dict_local[key]=value

    pred_test_dict_local = {}
    for key, value in pred_test_dict.items():
        if model in key:
            pred_test_dict_local[key]=value

    pred_dict_new_local = dict(sorted(
        pred_dict_local.items(), 
        key=lambda x:score_function((train_labels), np.argmax(list(x[1]), axis=1)), reverse=False)[:5])
    pred_test_dict_new_local = {}
    for key, value in pred_dict_new_local.items():
        pred_test_dict_new_local[key]=pred_test_dict_local[key]
        
    return pred_dict_new_local, pred_test_dict_new_local

In [ ]:
def save_dict(model, pred_dict, pred_test_dict):
    with open('../pickle/pred_train_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_dict, fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_test_dict, fw)

In [ ]:
pred_train_dict_global, pred_test_dict_global = sort_dict(name, pred_train_dict, pred_test_dict)
save_dict(name, pred_train_dict_global, pred_test_dict_global)